In [2]:
pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61395 sha256=fc9914a097b4be1af740724b021ea79ac53a309014162972bd88164ffd8b7db9
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31529 sha256=bd8fd41fbf09052877f720526a2e3c4935053ed369e15876ce113bb47156d93e
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
from fvcore.nn import FlopCountAnalysis
from collections import Counter

In [4]:
def calculate_flop_percentages(model, input_tensor):
    flop_analyzer = FlopCountAnalysis(model, input_tensor)
    flops_by_operator = flop_analyzer.by_operator()

    max_flop = max(flops_by_operator.values())
    table = []
    for operator, flop_count in flops_by_operator.items():
        relative_percentage = (flop_count / max_flop) * 100
        table.append((operator, flop_count, relative_percentage))

    return table

In [5]:
import torch
from torchvision.models import vgg19, resnet50, densenet121
from transformers import BertModel

In [6]:
input_tensor = torch.randn(1, 3, 224, 224)

In [7]:
vgg19_model = vgg19()
vgg19_table = calculate_flop_percentages(vgg19_model, input_tensor)

In [8]:
resnet50_model = resnet50()
resnet50_table = calculate_flop_percentages(resnet50_model, input_tensor)

In [9]:
densenet_model = densenet121()
densenet_table = calculate_flop_percentages(densenet_model, input_tensor)

In [10]:
bert_input_tensor = torch.randint(0, 10000, (1, 128))

In [11]:
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_table = calculate_flop_percentages(bert_model, bert_input_tensor)

# Print the results
for model_name, table in zip(["VGG19", "ResNet50", "DenseNet", "BERT-Uncased"],
                             [vgg19_table, resnet50_table, densenet_table, bert_table]):
    print(f"\nModel: {model_name}")
    print(f"{'Operator':<20} {'FLOP Count':<15} {'Relative %':<10}")
    for row in table:
        print(f"{row[0]:<20} {row[1]:<15} {row[2]:<10.2f}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

encoder.layer.0.attention.self.dropout, encoder.layer.1.attention.self.dropout, encoder.layer.10.attention.self.dropout, encoder.layer.11.attention.self.dropout, encoder.layer.2.attention.self.dropout, encoder.layer.3.attention.self.dropout, encoder.layer.4.attention.self.dropout, encoder.layer.5.attention.self.dropout, encoder.layer.6.attention.self.dropout, encoder.layer.7.attention.self.dropout, encoder.layer.8.attention.self.dropout, encoder.layer.9.attention.self.dropout



Model: VGG19
Operator             FLOP Count      Relative %
conv                 19508428800     100.00    
adaptive_avg_pool2d  25088           0.00      
linear               123633664       0.63      

Model: ResNet50
Operator             FLOP Count      Relative %
conv                 4087136256      100.00    
batch_norm           55569920        1.36      
adaptive_avg_pool2d  100352          0.00      
linear               2048000         0.05      

Model: DenseNet
Operator             FLOP Count      Relative %
conv                 2833137664      100.00    
batch_norm           78337280        2.77      
adaptive_avg_pool2d  50176           0.00      
linear               1024000         0.04      

Model: BERT-Uncased
Operator             FLOP Count      Relative %
layer_norm           12288000        0.11      
linear               10872225792     100.00    
